In [1]:
folder = 'two_motions'

In [2]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
%matplotlib ipympl

In [3]:
df_raw = pd.read_csv(os.path.join(folder, 'optomyo.txt'), sep = ' ', header=None, 
                     names=['timestamp'] + list(range(16)) + ['diode', 'trigger'],
                     skiprows=3)

In [4]:
df_raw['timestamp'] -= df_raw['timestamp'].iloc[0]
df_raw['time_s'] = df_raw['timestamp']/1000
df_raw['cycle'] = np.clip(np.abs(np.diff(df_raw.diode.values, prepend=df_raw.diode.iloc[0])), 0, 1).cumsum()
df_raw.drop_duplicates(subset=['cycle'], keep='last', inplace=True)
df_raw['full_cycle'] = df_raw.cycle//16

df_raw

,timestamp,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,diode,trigger,time_s,cycle,full_cycle
1,1,543,630,868,672,881,980,2054,18249,39852,...,914,800,909,745,679,8,0,0.001,0,0
3,2,680,678,667,709,788,751,1076,1674,27568,...,1158,719,845,763,1010,9,0,0.002,1,0
5,4,9307,1316,1942,806,1171,959,1062,1001,16253,...,21345,973,888,726,850,10,0,0.004,2,0
8,7,636,590,756,702,795,842,612,893,837,...,23611,14362,1043,777,806,11,0,0.007,3,0
10,9,639,741,747,705,820,880,938,948,716,...,14166,26954,19982,985,863,12,0,0.009,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409078,363455,466,288,486,513,602,1065,21232,34334,14946,...,381,544,528,434,511,7,0,363.455,193359,12084
409080,363457,479,361,392,402,445,818,3515,16711,39022,...,594,486,433,296,555,8,0,363.457,193360,12085
409082,363459,411,428,354,458,573,440,580,1490,24005,...,631,383,347,388,506,9,0,363.459,193361,12085
409084,363461,404,402,352,326,333,445,480,574,1558,...,19171,591,465,470,653,10,0,363.461,193362,12085


In [5]:
sanity_check = df_raw.groupby('diode').mean().iloc[:, np.arange(1,17)].values.argmax(axis=1)
print(sanity_check)
assert((sanity_check == np.arange(16)).all())
print('All good')

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
All good


In [6]:
df_mask = df_raw.groupby('diode').mean().iloc[:, np.arange(1,17)]
df_mask -= df_mask.median()
df_mask = df_mask >= df_mask.max(axis=1)/4

plt.figure()
plt.imshow(df_mask)
plt.title('Diode Mask')
df_mask

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
diode,,,,,,,,,,,,,,,,
0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False


In [7]:
def extract_channels(g):
    if len(g)<16:
        return None
    loc_time = g.time_s.mean()
    trigger = int(np.round(g.trigger.median()))
    df = g.sort_values(by='diode')\
          .iloc[:, np.arange(1,17)]
    channels = df.values[df_mask.values]
    return list(channels) + [loc_time, trigger]

In [8]:
df_opto = pd.DataFrame(np.stack(df_raw.groupby('full_cycle').apply(extract_channels)[:-1], axis=0), 
                       columns = list(np.arange(df_mask.values.reshape(-1).sum())) + ['time_s', 'trigger'])

df_opto.to_csv(os.path.join(folder, 'opto_preprocessed.csv'), sep='\t')